# 4.7 연습문제  
## 1. 모바일폰이나 카메라에 저장한 사진 중 빨간색, 파란색, 초록색 물건을 찍은 사진 몇 장을 준비해보자  
### a. 각 이미지를 읽어 텐서로 변환하라.

In [1]:
import imageio

red_1 = imageio.imread('red/01.jpg')
red_2 = imageio.imread('red/02.jpg')
red_3 = imageio.imread('red/03.jpg')

blue_1 = imageio.imread('blue/01.jpg')
blue_2 = imageio.imread('blue/02.jpg')
blue_3 = imageio.imread('blue/03.jpg')

green_1 = imageio.imread('green/01.jpg')
green_2 = imageio.imread('green/02.jpg')
green_3 = imageio.imread('green/03.jpg')


In [2]:
import torch

batch_size = 9
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [3]:
import numpy as np

imageArr = [red_1, red_2, red_3, blue_1, blue_2, blue_3, green_1, green_2, green_3]

for i, image in enumerate(imageArr):
    img_t = torch.from_numpy(image)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t

### b. 각 이미지 텐서에 대해 .mean() 메소드를 사용해 이미지가 얼마나 밝은지 확인하라.

In [4]:
batch = batch.float()
batch/=255.0

for i, image in enumerate(batch):
    print(image.mean())

tensor(0.3519)
tensor(0.7623)
tensor(0.2319)
tensor(0.6701)
tensor(0.2966)
tensor(0.7300)
tensor(0.3482)
tensor(0.6435)
tensor(0.3869)


### c. 이미지의 각 채널 평균을 구해보라. 채널 평균값만으로 빨간색, 파란색, 초록색 물건을 구별할 수 있는가?

In [5]:
n_channels = batch.shape[1]

for i, image in enumerate(batch):
    print(image.view(3,-1).mean(dim=1))

tensor([0.6023, 0.2187, 0.2346])
tensor([0.9260, 0.6772, 0.6836])
tensor([0.6493, 0.0272, 0.0193])
tensor([0.4860, 0.6783, 0.8459])
tensor([0.0736, 0.2525, 0.5637])
tensor([0.6606, 0.7279, 0.8014])
tensor([0.3463, 0.6158, 0.0825])
tensor([0.6124, 0.6984, 0.6196])
tensor([0.0840, 0.6053, 0.4713])


구별하기 어렵다


# 5.7 연습문제  
## 1. 모델을 w2 * t_u ** 2 + w1 * t_u + b로 다시 정의하자.


In [19]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, linewidth=75)

t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0,
                    3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u


In [8]:
def model(t_u, w, b):
    return w * t_u + b


In [9]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()


In [10]:
params = torch.tensor([1.0, 0.0], requires_grad=True)


In [11]:
params.grad is None


True

In [12]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()

params.grad


tensor([4517.2969,   82.6000])

In [13]:
if params.grad is not None:
    params.grad.zero_()


In [14]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:  # <1>
            params.grad.zero_()
        
        t_p = model(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        loss.backward()
        
        with torch.no_grad():  # <2>
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params


In [15]:
training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0, 0.0], requires_grad=True), # <1> 
    t_u = t_un, # <2> 
    t_c = t_c)


Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

### a. 이 모델을 다루기 위해 훈련 루프의 어떤 부분을 변경해야 하나?  
--> model 함수와 파라미터만 변경하면 된다.  
  
### b. 모델 변경과 무관한 부분은 어디인가?  
--> 훈련 루프

In [27]:
def model2(t_u, w1, w2, b):
    return w2 * t_u ** 2 + w1 * t_u + b

In [28]:
def training_loop2(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:  # <1>
            params.grad.zero_()
        
        t_p = model2(t_u, *params) 
        loss = loss_fn(t_p, t_c)
        loss.backward()
        
        with torch.no_grad():  # <2>
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params


In [65]:
training_loop2(
    n_epochs = 10000, 
    learning_rate = 1e-4, 
    params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True), # <1> 
    t_u = t_un, # <2> 
    t_c = t_c)

Epoch 500, Loss 10.708596
Epoch 1000, Loss 8.642083
Epoch 1500, Loss 7.171005
Epoch 2000, Loss 6.123477
Epoch 2500, Loss 5.377227
Epoch 3000, Loss 4.845285
Epoch 3500, Loss 4.465788
Epoch 4000, Loss 4.194724
Epoch 4500, Loss 4.000802
Epoch 5000, Loss 3.861744
Epoch 5500, Loss 3.761724
Epoch 6000, Loss 3.689469
Epoch 6500, Loss 3.636971
Epoch 7000, Loss 3.598528
Epoch 7500, Loss 3.570086
Epoch 8000, Loss 3.548757
Epoch 8500, Loss 3.532496
Epoch 9000, Loss 3.519839
Epoch 9500, Loss 3.509750
Epoch 10000, Loss 3.501493


tensor([-1.1696,  0.6077, -1.1362], requires_grad=True)

### c. 훈련 후 손실이 증가하는가 감소하는가?  
--> 손실이 감소하긴 하나, learning rate 수치를 조절하여야 한다.  
  
### d. 실제 결과가 좋아졌나 나빠졌나?  
--> 기존 모델보다 결과가 나빠졌다  